### Setup

In [1]:
# this first setup block will pretty much always stay the same
""" Python Standard Library """
import os
import datetime as dt
import json
import logging
""" Third Party Imports """
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
""" Local Imports """
if 'requirements.txt' not in os.listdir(os.getcwd()):
    os.chdir(os.path.join('..','..','..'))
print(os.getcwd())
from covid_model import CovidModel
from covid_model.runnable_functions import do_single_fit, do_create_multiple_reports, do_fit_scenarios
from covid_model.utils import setup, get_filepath_prefix
from covid_model.analysis.charts import plot_transmission_control

os.environ['gcp_project'] = 'co-covid-models'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "co-covid-models-credentials.json"

/Users/emwu9912/PycharmProjects/covid-models


In [2]:
# set up the output directory for this Jupyter notebook
outdir = setup("20220719_scenario_projections.ipynb")

2022/07/19 16:31:15 INFO|============================================================


### Fit an initial scenario through Feb 2022

In [ ]:
# designate the arguments for how the model will behave
model_args = {
    'params_defs': 'covid_model/analysis/20220628_july_gov_briefing/scenario_params.json',
    'region_defs': 'covid_model/input/region_definitions.json',
    'vacc_proj_params': 'covid_model/analysis/20220628_july_gov_briefing/vacc_proj_params.json',
    'start_date': '2020-01-24',
    'end_date': '2022-09-18',
}

# this is how the fit will behave
# place the outdir argument here to tell the model fit where to go
fit_args = {'outdir': outdir,
            'fit_end_date': '2022-02-28'
}

# because all the scenarios are the same 
model = do_single_fit(**fit_args, **model_args)

### Create and run scenarios from Feb 2022 to present

In [9]:
multiprocess = 6

scenario_params = json.load(open("covid_model/analysis/20220628_july_gov_briefing/scenario_params.json"))

model_args = {
    'base_spec_id': 3214 #model.spec_id # use the spec id that was output from the model fit
}
model_fit_args = {
    'outdir': outdir,
    'fit_start_date': '2022-03-01', # set the start date for the earliest point at which the scenarios start to differ from one another
    'pre_solve_model': True # force the model to establish initial conditions so the fit can start on the fit start date
}

# compute vacc effectiveness for < 5 (this is a multiplier for the baseline vaccine effectiveness for 0-19)
vacc_eff_lt5 = 0.5

# Create different scenarios to model
scenario_model_args = []
for hosp_frac in [0.5, 0.7, 0.9]:
    for school_contact_increase in [1.1, 1.2]:
        for beta_mult in [1.0, 1.1]:
            hrf = {"2020-01-01": 1, "2022-03-01": (0.66 + 0.34*hosp_frac), "2022-03-15": (0.34 + 0.66*hosp_frac), "2022-03-30": hosp_frac}
            beta_param_adjustment = [{"param": "betta",
                                      "attrs": {"variant": "ba45"},
                                      "mults": {"2020-01-01": beta_mult},
                                      "desc": "sensitivity analysis for beta"}]
            school_contact = [{"param": "betta",
                               "attrs": {'age': '0-19'},
                               "mults": {"2020-01-01": 1, "2022-08-15": school_contact_increase},
                               "desc": "After August 15th, beta increases by a factor of the school contact increase"}]
            lt5_vacc_adjust = [{"param": "immunity",
                                "attrs": {'age': '0-19', 'vacc': 'shot1'},
                                "mults": {"2020-01-01": 1, "2022-06-20": 0.99 + 0.01*vacc_eff_lt5,
                                          "2022-06-30": 0.98 + 0.02*vacc_eff_lt5, "2022-07-10": 0.97 + 0.03*vacc_eff_lt5}}]
            scenario_model_args.append({'params_defs': scenario_params + beta_param_adjustment + school_contact + lt5_vacc_adjust,
                                        'hosp_reporting_frac': hrf,
                                        'tags': {'beta_mult': beta_mult, 'hosp_frac': hosp_frac, 'school_contact_increase': school_contact_increase}})
            
            

In [ ]:
# check how many scenarios there are
len(scenario_model_args)

In [11]:
# run the scenarios
models = do_fit_scenarios(base_model_args=model_args, scenario_args_list=scenario_model_args, fit_args=model_fit_args, multiprocess=multiprocess)

2022/07/19 16:35:18 INFO|============================================================
2022/07/19 16:35:18 INFO|============================================================
2022/07/19 16:35:18 INFO|============================================================
2022/07/19 16:35:18 INFO|============================================================
2022/07/19 16:35:18 INFO|============================================================
2022/07/19 16:35:18 INFO|============================================================
2022/07/19 16:35:21 INFO|-----------|{'beta_mult': 1.0, 'hosp_frac': 0.7, 'school_contact_increase': 1.1} Retrieving vaccinations data
2022/07/19 16:35:21 INFO|-----------|{'beta_mult': 1.0, 'hosp_frac': 0.5, 'school_contact_increase': 1.2} Retrieving vaccinations data
2022/07/19 16:35:21 INFO|-----------|{'beta_mult': 1.0, 'hosp_frac': 0.5, 'school_contact_increase': 1.1} Retrieving vaccinations data
2022/07/19 16:35:21 INFO|-----------|{'beta_mult': 1.1, 'hosp_frac': 0.5, 'scho

2022/07/19 16:36:55 INFO|--------|{'beta_mult': 1.1, 'hosp_frac': 0.7, 'school_contact_increase': 1.1, 'fit_batch': '0'}: Transmission control fit 1/3 completed in 13.871596291000003 seconds: {767: {'co': 0.7897954843119923}, 781: {'co': 0.8322022199388147}, 795: {'co': 0.7994378902490691}, 809: {'co': 0.81347638163633}, 823: {'co': 0.819233758990234}, 837: {'co': 0.818080242250903}}
2022/07/19 16:36:55 INFO|---------|{'beta_mult': 1.1, 'hosp_frac': 0.7, 'school_contact_increase': 1.1, 'fit_batch': '0'}: Running forward sim
2022/07/19 16:36:56 INFO|--------|{'beta_mult': 1.0, 'hosp_frac': 0.5, 'school_contact_increase': 1.1, 'fit_batch': '0'}: Transmission control fit 1/3 completed in 13.798235334000012 seconds: {767: {'co': 0.7708318945448158}, 781: {'co': 0.8174501531407756}, 795: {'co': 0.7963881049040153}, 809: {'co': 0.8048672667610491}, 823: {'co': 0.8059913601692489}, 837: {'co': 0.7949812191259992}}
2022/07/19 16:36:56 INFO|---------|{'beta_mult': 1.0, 'hosp_frac': 0.5, 'school

2022/07/19 16:37:22 INFO|--------|{'beta_mult': 1.1, 'hosp_frac': 0.5, 'school_contact_increase': 1.2, 'fit_batch': '2'}: Transmission control fit 3/3 completed in 13.683887667000008 seconds: {809: {'co': 0.806860852113268}, 823: {'co': 0.8029086058083854}, 837: {'co': 0.8102479983439306}, 851: {'co': 0.7994520341946584}, 865: {'co': 0.8392456221614089}, 879: {'co': 0.8085423050144604}}
2022/07/19 16:37:22 INFO|---------|{'beta_mult': 1.1, 'hosp_frac': 0.5, 'school_contact_increase': 1.2, 'fit_batch': '2'}: Running forward sim
2022/07/19 16:37:23 INFO|--------|{'beta_mult': 1.0, 'hosp_frac': 0.5, 'school_contact_increase': 1.1, 'fit_batch': '2', 'run_type': 'fit'}: fitted TC: {0: {'co': 0.8773342495490936}, 14: {'co': 0.6853524265751645}, 28: {'co': 0.5313747878505213}, 42: {'co': 9.166775206693499e-14}, 56: {'co': 0.7004806299042426}, 70: {'co': 0.8356320023671157}, 84: {'co': 0.8298063033343771}, 98: {'co': 0.8311909839169287}, 112: {'co': 0.8778137464237421}, 126: {'co': 0.824110375

2022/07/19 16:37:25 INFO|============================================================
2022/07/19 16:37:26 INFO|--------|{'beta_mult': 1.1, 'hosp_frac': 0.5, 'school_contact_increase': 1.1, 'fit_batch': '2', 'run_type': 'fit'}: fitted TC: {0: {'co': 0.8773342495490936}, 14: {'co': 0.6853524265751645}, 28: {'co': 0.5313747878505213}, 42: {'co': 9.166775206693499e-14}, 56: {'co': 0.7004806299042426}, 70: {'co': 0.8356320023671157}, 84: {'co': 0.8298063033343771}, 98: {'co': 0.8311909839169287}, 112: {'co': 0.8778137464237421}, 126: {'co': 0.8241103755501108}, 140: {'co': 0.7606632825445033}, 154: {'co': 0.6711880168053298}, 168: {'co': 0.8430307682247831}, 182: {'co': 0.7826561801378445}, 196: {'co': 0.8191830543380944}, 210: {'co': 0.7726034259026388}, 224: {'co': 0.7713676622018327}, 238: {'co': 0.7185596188624429}, 252: {'co': 0.7222209085605668}, 266: {'co': 0.6848822730370463}, 280: {'co': 0.7199443312231306}, 294: {'co': 0.7575590209953923}, 308: {'co': 0.8226615649781124}, 322: {'c

2022/07/19 16:37:35 INFO|============================================================
2022/07/19 16:37:36 INFO|--------|{'beta_mult': 1.0, 'hosp_frac': 0.9, 'school_contact_increase': 1.2} Prepping Model
2022/07/19 16:37:36 INFO|---------|{'beta_mult': 1.0, 'hosp_frac': 0.9, 'school_contact_increase': 1.2} Prepping Model
2022/07/19 16:37:37 INFO|-----------|{'beta_mult': 1.1, 'hosp_frac': 0.9, 'school_contact_increase': 1.2} Retrieving vaccinations data
2022/07/19 16:37:38 INFO|-----------|{'beta_mult': 1.1, 'hosp_frac': 0.9, 'school_contact_increase': 1.2} Constructing vaccination projections
2022/07/19 16:37:38 INFO|-----------|{'beta_mult': 1.1, 'hosp_frac': 0.9, 'school_contact_increase': 1.2} Retrieving hospitalizations data
2022/07/19 16:37:39 INFO|--------|{'beta_mult': 1.1, 'hosp_frac': 0.9, 'school_contact_increase': 1.2} Prepping Model
2022/07/19 16:37:39 INFO|---------|{'beta_mult': 1.1, 'hosp_frac': 0.9, 'school_contact_increase': 1.2} Prepping Model
2022/07/19 16:38:29 INF

2022/07/19 16:39:06 INFO|--------|{'beta_mult': 1.0, 'hosp_frac': 0.9, 'school_contact_increase': 1.1, 'fit_batch': '1'}: Transmission control fit 2/3 completed in 13.560958916999994 seconds: {795: {'co': 0.8017296950466122}, 809: {'co': 0.8209654561199187}, 823: {'co': 0.8219589991065116}, 837: {'co': 0.8297913198659961}, 851: {'co': 0.8215177744214113}, 865: {'co': 0.8531226711758313}}
2022/07/19 16:39:06 INFO|---------|{'beta_mult': 1.0, 'hosp_frac': 0.9, 'school_contact_increase': 1.1, 'fit_batch': '1'}: Running forward sim
2022/07/19 16:39:06 INFO|--------|{'beta_mult': 1.1, 'hosp_frac': 0.9, 'school_contact_increase': 1.1, 'fit_batch': '1'}: Transmission control fit 2/3 completed in 13.354357417000017 seconds: {795: {'co': 0.8017165910167425}, 809: {'co': 0.8210041360363891}, 823: {'co': 0.8235314258119252}, 837: {'co': 0.8354010157480687}, 851: {'co': 0.8356450108118318}, 865: {'co': 0.8701020298706122}}
2022/07/19 16:39:06 INFO|---------|{'beta_mult': 1.1, 'hosp_frac': 0.9, 'sc

2022/07/19 16:39:26 INFO|--------|{'beta_mult': 1.1, 'hosp_frac': 0.9, 'school_contact_increase': 1.1, 'fit_batch': '2', 'run_type': 'fit'}: fitted TC: {0: {'co': 0.8773342495490936}, 14: {'co': 0.6853524265751645}, 28: {'co': 0.5313747878505213}, 42: {'co': 9.166775206693499e-14}, 56: {'co': 0.7004806299042426}, 70: {'co': 0.8356320023671157}, 84: {'co': 0.8298063033343771}, 98: {'co': 0.8311909839169287}, 112: {'co': 0.8778137464237421}, 126: {'co': 0.8241103755501108}, 140: {'co': 0.7606632825445033}, 154: {'co': 0.6711880168053298}, 168: {'co': 0.8430307682247831}, 182: {'co': 0.7826561801378445}, 196: {'co': 0.8191830543380944}, 210: {'co': 0.7726034259026388}, 224: {'co': 0.7713676622018327}, 238: {'co': 0.7185596188624429}, 252: {'co': 0.7222209085605668}, 266: {'co': 0.6848822730370463}, 280: {'co': 0.7199443312231306}, 294: {'co': 0.7575590209953923}, 308: {'co': 0.8226615649781124}, 322: {'co': 0.8113711381077103}, 336: {'co': 0.7825940193755743}, 350: {'co': 0.79524722351915

2022/07/19 16:40:38 INFO|-----------------------|spec_ids: 3458,3459,3460,3461,3462,3463,3464,3465,3466,3467,3468,3469


### Run the report for each fit model

In [12]:
do_create_multiple_reports(models, multiprocess=multiprocess, outdir=outdir, prep_model=False, solve_model=True, immun_variants=['ba2121', 'ba45'], from_date='2022-01-01')

2022/07/19 16:40:53 INFO|============================================================
2022/07/19 16:40:53 INFO|--------|Solving model
2022/07/19 16:40:54 INFO|============================================================
2022/07/19 16:40:54 INFO|--------|Solving model
2022/07/19 16:40:54 INFO|============================================================
2022/07/19 16:40:54 INFO|--------|Solving model
2022/07/19 16:40:55 INFO|============================================================
2022/07/19 16:40:55 INFO|--------|Solving model
2022/07/19 16:40:56 INFO|============================================================
2022/07/19 16:40:56 INFO|--------|Solving model
2022/07/19 16:40:56 WARNING|No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
2022/07/19 16:40:57 WARNING|No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored wh

In [13]:
logging.info('Projecting')
for model in models:
    logging.info('')
    #model.prep()  # don't think we need to prep anymore.
    model.solve_seir()

    model.solution_sum_df(['seir', 'variant', 'immun']).unstack().to_csv(get_filepath_prefix(outdir, tags=model.tags) + 'states_seir_variant_immun_total_all_at_once_projection.csv')
    model.solution_sum_df().unstack().to_csv(get_filepath_prefix(outdir, tags=model.tags) + 'states_full_projection.csv')

    logging.info(f'{str(model.tags)}: Running forward sim')
    fig = plt.figure(figsize=(10, 10), dpi=300)
    ax = fig.add_subplot(211)
    hosps_df = model.modeled_vs_observed_hosps().reset_index('region').drop(columns='region')
    hosps_df.plot(ax=ax)
    ax.set_xlim(dt.datetime.strptime('2022-01-01', "%Y-%m-%d").date(), dt.datetime.strptime('2022-09-15', "%Y-%m-%d").date())
    ax = fig.add_subplot(212)
    plot_transmission_control(model, ax=ax)
    ax.set_xlim(dt.datetime.strptime('2022-01-01', "%Y-%m-%d").date(), dt.datetime.strptime('2022-09-15', "%Y-%m-%d").date())
    plt.savefig(get_filepath_prefix(outdir, tags=model.tags) + 'model_forecast.png')
    plt.close()
    hosps_df.to_csv(get_filepath_prefix(outdir, tags=model.tags) + '_model_forecast.csv')
    json.dump(model.tc, open(get_filepath_prefix(outdir, tags=model.tags) + 'model_forecast_tc.json', 'w'))

logging.info('Running reports')

2022/07/19 16:46:32 INFO|Projecting
2022/07/19 16:46:32 INFO|
2022/07/19 16:46:41 INFO|{'beta_mult': 1.0, 'hosp_frac': 0.5, 'school_contact_increase': 1.1, 'fit_batch': '2', 'run_type': 'fit'}: Running forward sim
2022/07/19 16:46:42 INFO|
2022/07/19 16:46:56 INFO|{'beta_mult': 1.1, 'hosp_frac': 0.5, 'school_contact_increase': 1.1, 'fit_batch': '2', 'run_type': 'fit'}: Running forward sim
2022/07/19 16:46:57 INFO|
2022/07/19 16:47:06 INFO|{'beta_mult': 1.0, 'hosp_frac': 0.5, 'school_contact_increase': 1.2, 'fit_batch': '2', 'run_type': 'fit'}: Running forward sim
2022/07/19 16:47:07 INFO|
2022/07/19 16:47:21 INFO|{'beta_mult': 1.1, 'hosp_frac': 0.5, 'school_contact_increase': 1.2, 'fit_batch': '2', 'run_type': 'fit'}: Running forward sim
2022/07/19 16:47:22 INFO|
2022/07/19 16:47:31 INFO|{'beta_mult': 1.0, 'hosp_frac': 0.7, 'school_contact_increase': 1.1, 'fit_batch': '2', 'run_type': 'fit'}: Running forward sim
2022/07/19 16:47:32 INFO|
2022/07/19 16:47:45 INFO|{'beta_mult': 1.1, 'hos